In [80]:
import json
import requests
import time
import pandas as pd

import os
from os.path import join, dirname
from dotenv import load_dotenv


dotenv_path = join(dirname(os.path.dirname('__file__')), '.env')
load_dotenv(dotenv_path)

from sqlalchemy import create_engine

# build connecting string from .env
DATABASE_NAME = "oxcart" #os.getenv("BIG_BOY_DB_NAME")
DATABASE_USER = os.getenv("BIG_BOY_DB_USER")
DATABASE_PASSWORD = os.getenv("BIG_BOY_DB_PASSWORD")
DATABASE_HOST = os.getenv("BIG_BOY_DB_HOST")
DATABASE_PORT = os.getenv("BIG_BOY_DB_PORT")

connection = "postgresql://"+DATABASE_USER+":"+DATABASE_PASSWORD+"@"+DATABASE_HOST+":"+DATABASE_PORT+"/"+DATABASE_NAME

conn = create_engine(connection) 

In [65]:
# get arbitrum top delegates
top_delegates_query = """
    SELECT * FROM arbitrum.top_delegates WHERE delegate_rank is not null;
"""
top_delegates_df = pd.read_sql(top_delegates_query, conn)

In [4]:
top_delegates_df

,address,name,vp,delegate_rank
0,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,delegate.l2beat.eth,18838105,1
1,0xf4b0556b9b6f53e00a1fdd2b0478ce841991d8fa,olimpio.eth,15358347,3
2,0xb933aee47c438f22de0747d57fc239fe37878dd1,wintermutegovernance.eth,14405315,4
3,0x0eb5b03c0303f2f47cd81d7be4275af8ed347576,TreasureDAO,14108091,5
4,0xf92f185abd9e00f56cb11b0b709029633d1e37b4,Coinflipcanada,12471044,6
...,...,...,...,...
95,0xa71a021ef66b03e45e0d85590432dfcfa1b7174c,None,931604,52
96,0xdd5905172a02fa66b45188ff70f03cca0628164d,None,710266,54
97,0x5180db0237291a6449dda9ed33ad90a38787621c,None,635609,56
98,0xb4c064f466931b8d0f637654c916e3f203c46f13,None,517730,64


In [66]:
proposal_query = "SELECT * FROM snapshot.proposallist WHERE dao_id = 'arbitrumfoundation.eth'"
proposal_df = pd.read_sql_query(proposal_query, conn)
arbitrum_votes = f"""
    SELECT * from snapshot.votelist where proposal IN {tuple(proposal_df['proposal_id'])} AND lower(voter) IN {tuple(top_delegates_df['address'])};
"""
arbitrum_votes_df = pd.read_sql_query(arbitrum_votes, conn)
# remove vote_id, app, reason, typename
arbitrum_votes_df = arbitrum_votes_df.drop(columns=['vote_id', 'app', 'reason', 'typename'])
# join proposals on proposal and proposal_id
arbitrum_votes_and_proposals = pd.merge(arbitrum_votes_df, proposal_df, left_on='proposal', right_on='proposal_id')
# drop proposal_id, plugins, discussion, ipfs, app, link, privacy, flagged
arbitrum_votes_and_proposals = arbitrum_votes_and_proposals.drop(columns=['proposal_id', 'plugins', 'discussion', 'ipfs', 'app', 'link', 'privacy', 'flagged'])
# drop created_x; rename created_y to created
arbitrum_votes_and_proposals = arbitrum_votes_and_proposals.drop(columns=['created_x'])
arbitrum_votes_and_proposals = arbitrum_votes_and_proposals.rename(columns={'created_y':'created'})
# clone df with proposal_id, voter, vp, choice, created
arbitrum_votes_and_proposals_clone = arbitrum_votes_and_proposals.copy()
arb_votes_shallow = arbitrum_votes_and_proposals_clone[['proposal', 'voter', 'vp', 'choice', 'created']]
arb_votes_shallow

,proposal,voter,vp,choice,created
0,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,0x3070FeEAfD678751c29171Bacd0042Fb370DEE00,3.183737e+06,1,1.691510e+09
1,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,0x190473B3071946df65306989972706A4c006A561,1.061807e+07,1,1.691510e+09
2,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,2.779683e+06,1,1.691510e+09
3,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,1.225209e+07,1,1.691510e+09
4,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,0xB933AEe47C438f22DE0747D57fc239FE37878Dd1,1.500001e+06,1,1.691510e+09
...,...,...,...,...,...
7251,0x5e43fdcaa085476a7be4fb638dd57f7b3f6b708fbbbe...,0xd333Bc5c9670C9cEb18f9A2CF02C6E86807a8227,4.152247e+06,1,1.689965e+09
7252,0x5e43fdcaa085476a7be4fb638dd57f7b3f6b708fbbbe...,0xE93D59CC0bcECFD4ac204827eF67c5266079E2b5,6.549522e+05,1,1.689965e+09
7253,0x5e43fdcaa085476a7be4fb638dd57f7b3f6b708fbbbe...,0x13BDaE8c5F0fC40231F0E6A4ad70196F59138548,7.625152e+05,1,1.689965e+09
7254,0x5e43fdcaa085476a7be4fb638dd57f7b3f6b708fbbbe...,0xea172676E4105e92Cc52DBf45fD93b274eC96676,4.587794e+05,1,1.689965e+09


In [67]:
from scipy.stats import spearmanr
from collections.abc import Iterable

def normalize_choices(choices, all_choices):
    max_rank = len(all_choices)
    normalized = [max_rank + 1] * max_rank

    for idx, choice in enumerate(choices):
        if choice in all_choices:
            normalized[all_choices.index(choice)] = idx + 1

    return normalized

def handle_object(obj):
    sorted_items = sorted(obj.items(), key=lambda item: item[1])
    return [int(key) for key, _ in sorted_items]

def calculate_similarity_score(choices1, choices2):
    if isinstance(choices1, int) and isinstance(choices2, int):
        return 1.0 if choices1 == choices2 else 0.0

    if isinstance(choices1, list) and isinstance(choices2, list):
        all_choices = sorted(set(choices1) | set(choices2))
        norm_choices1 = normalize_choices(choices1, all_choices)
        norm_choices2 = normalize_choices(choices2, all_choices)
        correlation, _ = spearmanr(norm_choices1, norm_choices2)
        
        if np.isnan(correlation):
            correlation = 1.0 if norm_choices1 == norm_choices2 else 0.0
        
        return (correlation + 1) / 2

    if isinstance(choices1, dict) and isinstance(choices2, dict):
        choices1_list = handle_object(choices1)
        choices2_list = handle_object(choices2)
        all_choices = sorted(set(choices1_list) | set(choices2_list))
        norm_choices1 = normalize_choices(choices1_list, all_choices)
        norm_choices2 = normalize_choices(choices2_list, all_choices)
        correlation, _ = spearmanr(norm_choices1, norm_choices2)
        
        if np.isnan(correlation):
            correlation = 1.0 if norm_choices1 == norm_choices2 else 0.0
        
        return (correlation + 1) / 2

    return 0.0


In [72]:
import pandas as pd
import numpy as np
from itertools import combinations

def calculate_voter_similarity(data):
    grouped = data.groupby('proposal')
    
    voter_similarity = {}

    for proposal, group in grouped:
        voters = group['voter'].tolist()
        choices = group['choice'].tolist()
        
        for (i, voter1), (j, voter2) in combinations(enumerate(voters), 2):
            score = calculate_similarity_score(choices[i], choices[j])
            
            if voter1 not in voter_similarity:
                voter_similarity[voter1] = {}
            if voter2 not in voter_similarity[voter1]:
                voter_similarity[voter1][voter2] = 0
            voter_similarity[voter1][voter2] += score
            
            if voter2 not in voter_similarity:
                voter_similarity[voter2] = {}
            if voter1 not in voter_similarity[voter2]:
                voter_similarity[voter2][voter1] = 0
            voter_similarity[voter2][voter1] += score

    most_similar_voter = []
    for voter, similarities in voter_similarity.items():
        most_similar_voter_name = max(similarities, key=similarities.get)
        most_similar_voter_score = similarities[most_similar_voter_name]
        most_similar_voter.append({
            'voter': voter,
            'most_similar_voter': most_similar_voter_name,
            'similarity_score': most_similar_voter_score
        })

    most_similar_voter_df = pd.DataFrame(most_similar_voter)
    return [voter_similarity, most_similar_voter_df]

In [73]:
similarities_df, most_similar_voter_df = calculate_voter_similarity(arb_votes_shallow)

In [81]:
# dataframe of similartiies_df
similarities_df_df = pd.DataFrame(similarities_df)
# convert to df to have the rows delegate and columns delegate
similarities_df_df = similarities_df_df.reset_index().melt(id_vars='index')
# replace NaN in value with 0
similarities_df_df['value'] = similarities_df_df['value'].fillna(0)
#rename columns index->delegate, variable->compared_delegate, value->similarity_score
similarities_df_df = similarities_df_df.rename(columns={'index':'delegate', 'variable':'compared_delegate', 'value':'similarity_score'})
similarities_df_df
similarities_df_df.to_sql('top_delegate_similarity_full_mapping', conn, if_exists='replace', index=False)

561

In [70]:
# order similarities by similarity_score
similarities = most_similar_voter_df.sort_values(by=['similarity_score'], ascending=False)
similarities


,voter,most_similar_voter,similarity_score
5,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,160.121194
23,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,160.121194
17,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,0x18BF1a97744539a348304E9d266aAc7d446a1582,152.563636
30,0x18BF1a97744539a348304E9d266aAc7d446a1582,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,152.563636
4,0xa6e8772af29b29B9202a073f8E36f447689BEef6,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,150.512062
...,...,...,...
79,0xD4aA67f778Bba2805bC122F18F3d61Df59C7A542,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000
77,0x98033EFB9125333f4F6353BcC5e2D0Ae06b436a2,0x5f38BB373dccB91AD9Fd3727C2b9BaF6DF9332D3,1.000000
74,0xc3A48B40b3762924D6fa3af1D957cE78E522497E,0x47E98aA12dB2017295d33e94002521820442F827,1.000000
66,0xdd5905172A02FA66B45188Ff70F03CcA0628164d,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000


In [53]:
# rename voter to delegate
# rename most_similar_voter to most_similar_delegate
similarities = similarities.rename(columns={'voter':'delegate', 'most_similar_voter':'most_similar_delegate'})

In [56]:
# get count of votes each delegate has made
votes_per_delegate = arb_votes_shallow['voter'].value_counts().reset_index()
votes_per_delegate.columns = ['delegate', 'votes']
votes_per_delegate

,delegate,votes
0,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,248
1,0x6f9BB7e454f5B3eb2310343f0E99269dC2BB8A1d,241
2,0x5f38BB373dccB91AD9Fd3727C2b9BaF6DF9332D3,230
3,0xea172676E4105e92Cc52DBf45fD93b274eC96676,216
4,0xAD16ebE6FfC7d96624A380F394cD64395B0C6144,214
...,...,...
76,0xBA875DED061896896654f87E3B0bb908232cd4ea,1
77,0xD4aA67f778Bba2805bC122F18F3d61Df59C7A542,1
78,0xdd5905172A02FA66B45188Ff70F03CcA0628164d,1
79,0x98033EFB9125333f4F6353BcC5e2D0Ae06b436a2,1


In [54]:
similarities

,delegate,most_similar_delegate,similarity_score
5,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,160.121194
23,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,160.121194
17,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,0x18BF1a97744539a348304E9d266aAc7d446a1582,152.563636
30,0x18BF1a97744539a348304E9d266aAc7d446a1582,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,152.563636
4,0xa6e8772af29b29B9202a073f8E36f447689BEef6,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,150.512062
...,...,...,...
74,0xc3A48B40b3762924D6fa3af1D957cE78E522497E,0x47E98aA12dB2017295d33e94002521820442F827,1.000000
79,0xD4aA67f778Bba2805bC122F18F3d61Df59C7A542,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000
77,0x98033EFB9125333f4F6353BcC5e2D0Ae06b436a2,0x5f38BB373dccB91AD9Fd3727C2b9BaF6DF9332D3,1.000000
66,0xdd5905172A02FA66B45188Ff70F03CcA0628164d,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000


In [57]:
# similarity_score_percent is similarity_score / votes_per_delegate[delegate].votes_per_delegate
similarities = pd.merge(similarities, votes_per_delegate, on='delegate')
similarities['similarity_score_percent'] = similarities['similarity_score'] / similarities['votes']
similarities

,delegate,most_similar_delegate,similarity_score,votes,similarity_score_percent
0,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,160.121194,248,0.645650
1,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,160.121194,202,0.792679
2,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,0x18BF1a97744539a348304E9d266aAc7d446a1582,152.563636,198,0.770523
3,0x18BF1a97744539a348304E9d266aAc7d446a1582,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,152.563636,185,0.824668
4,0xa6e8772af29b29B9202a073f8E36f447689BEef6,0xAbAbE91A907fAdff921720C43ABe4098e7245E99,150.512062,206,0.730641
...,...,...,...,...,...
76,0xc3A48B40b3762924D6fa3af1D957cE78E522497E,0x47E98aA12dB2017295d33e94002521820442F827,1.000000,1,1.000000
77,0xD4aA67f778Bba2805bC122F18F3d61Df59C7A542,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000,1,1.000000
78,0x98033EFB9125333f4F6353BcC5e2D0Ae06b436a2,0x5f38BB373dccB91AD9Fd3727C2b9BaF6DF9332D3,1.000000,1,1.000000
79,0xdd5905172A02FA66B45188Ff70F03CcA0628164d,0xf3FE8c6c75bE4afB2F8200Fc77339abE4D7CFF33,1.000000,1,1.000000


In [61]:
# store to arbitrum.top_delegate_similarity
similarities.to_sql('top_delegate_similarity', conn, if_exists='replace', index=False)


81